# ETL Project
## Irvine Company Apartments


In [8]:
# set environment
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

from sqlalchemy import create_engine, insert
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Tolga\.wdm\drivers\chromedriver\win32\88.0.4324.96]


## Build URL for scraping cost-of-living index
- Near-future release: Use drop down for chosing region and date

In [3]:
# Base url for cost of living index by city from regional rankings
base_url = "https://www.numbeo.com/cost-of-living/region_rankings.jsp"

# Choose region
region_idx = 21   # 21 is the North America region
# Date
year = 2021       # Year of the acquisition
mid_year = False  # Not sure what this means, but there's an option to choose

In [4]:
# Date string
date_str = "%d" % (year)
if mid_year:
    date_str += "-mid"

# Region string
region_str = "%03d" % (region_idx)

# build URL for scraping 
cost_of_living_idx_url = base_url + f"?title={date_str}&region={region_str}"

In [5]:
browser.visit(cost_of_living_idx_url)
html = browser.html
# soup = BeautifulSoup(html, "html.parser")


In [10]:
rankings_df = pd.read_html(html)[1]
rankings_df.set_index("Rank", inplace=True)
# rankings_df.to_csv("data/cost_of_living.csv")
address_df = rankings_df["City"].str.split(", ", expand=True)
rankings_df["City"] = address_df[0]
rankings_df["State"] = address_df[1]

# Keep only the cities inside United States
rankings_df = rankings_df[address_df[2] == "United States"]
rankings_df_CA = rankings_df[rankings_df["State"] == "CA"]
rankings_df_CA

,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,State
Rank,,,,,,,,
4,San Francisco,93.35,109.35,100.99,93.50,90.85,126.95,CA
6,Oakland,89.59,84.14,86.98,94.96,74.55,92.30,CA
14,Los Angeles,81.92,74.12,78.19,79.26,89.11,98.51,CA
15,Fremont,81.77,74.04,78.08,80.17,84.16,121.55,CA
20,San Diego,80.22,67.85,74.31,80.28,78.70,112.72,CA
25,Irvine,79.21,69.90,74.76,72.42,79.58,126.46,CA
28,San Jose,78.55,82.92,80.64,74.99,78.75,131.28,CA
31,Sacramento,76.73,47.41,62.73,75.18,68.26,101.99,CA
51,Long Beach,73.43,56.12,65.16,77.76,67.76,106.23,CA
